# AI Dev Explainer - AI is stateless

This is a notebook for https://www.aiexplainer.dev/stateless

View all notebooks at https://github.com/chroma-core/ai_explainer

## Get an API Key from Anthropic
1. Create an account - https://console.anthropic.com/
2. Create an API key - https://console.anthropic.com/settings/keys
3. Replace `<your_api_key>` below

## Run the code
In Colab - you can click, "Runtime -> "Run All" in the topbar

In VS Code - you can click "Run All" in the topbar

In [ ]:
# install dependencies
! pip install anthropic

In [ ]:

import os
import anthropic

# Initialize the Anthropic client
# Replace '<your_api_key>' with your actual Anthropic API key
client = anthropic.Client(api_key='<your_api_key>')

alert = "Slow query performance detected on database-01. Average query time: 2.5 seconds"

messages = [
    {
        "role": "user",
        "content": alert
    }
]

response = client.messages.create(
    system="You are an AI assistant monitoring database performance. When you receive an alert, gather diagnostic information and generate commands to optimize the database. Send only 1 command that I shoud run and send only the command with no comments before or after.",
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=messages
)

print("Diagnostic command:")
print(response.content[0].text)

# Let's assume it asks the users to run
# #!/bin/bash
# # Get top CPU-consuming processes
# ssh admin@web-server-01 "top -b -n 1 -o %CPU | head -n 5"
# # Check system load average
# ssh admin@web-server-01 "uptime"

# Simulate running the diagnostic commands
diagnostic_results = """
Top 5 slow queries:
1. SELECT * FROM orders WHERE date > '2023-01-01' (avg time: 3.2s)
2. SELECT customers.*, orders.* FROM customers JOIN orders ON customers.id = orders.customer_id (avg time: 2.8s)
3. UPDATE inventory SET stock = stock - 1 WHERE product_id IN (SELECT product_id FROM order_items WHERE order_id = 12345) (avg time: 2.5s)
4. SELECT COUNT(*) FROM log_entries WHERE timestamp > NOW() - INTERVAL 1 DAY (avg time: 2.3s)
5. DELETE FROM expired_sessions WHERE last_activity < NOW() - INTERVAL 30 DAY (avg time: 2.1s)

Current database stats:
- Buffer pool hit ratio: 85%
- Index usage: 70%
- Table scans: 25% of queries
- Temporary tables created on disk: 15% of queries
"""

# Add the diagnostic results to the messages
messages.append({
    "role": "assistant",
    "content": response.content[0].text
})
messages.append({
    "role": "user",
    "content": diagnostic_results
})

# Get optimization commands from the LLM
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=messages
)

print("\nOptimization commands:")
print(response.content[0].text)